# Segment and Cluster Toronto Neighborhoods

## Part 1: Scraping Wikipedia Data

In [8]:
!conda install -c anaconda xlrd --yes
!conda install -c anaconda beautifulsoup4

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be SUPERSEDED by a higher-priority channel:

    xlrd: 1.1.0-py35h45a0a2a_1 --> 1.1.0-py35_1 anaconda

xlrd-1.1.0-py3 100% |################################| Time: 0:00:00  40.53 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0 anaconda

beautifulsoup4 100% |################################| Time: 0:00:00  39.53 MB/s


In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors

Fetch the page content from Wikipedia

In [10]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
contents = page.content

NOTE:

In the code below I make the following assumptions:

1. The table I'm looking has a specific class 'wikitable' and that there is only 1 table with that class. 
2. The data in the table is displayed in the order Postalcode, Borough, Neighborhood.
3. All tables have a value (including "Not assigned")

Having inspected the HTML of the page I clarified that the above assumptions were true at the time of creation (March 16th, 2019)

----

Use BeautifulSoup to help scrape data from the returned Wikipedia page content

Docs: https://www.crummy.com/software/BeautifulSoup/

In [11]:
soup = BeautifulSoup(contents, 'html.parser')

headers = ['Postcode', 'Borough', 'Neighborhood']

table = soup.find('table',{'class':'wikitable'})
table_rows = table.find_all('tr')
table_rows = table_rows[1:]

df_rows = []

for row in table_rows:
    items = row.find_all('td')
    if items[1].text.strip() != 'Not assigned':
        df_row =[]
        df_row.append(items[0].text.strip())
        df_row.append(items[1].text.strip())
        df_row.append(items[2].text.strip())
        df_rows.append(df_row)


In [12]:
init_df = pd.DataFrame(data=df_rows, columns=headers)
init_df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Now that we have the initial dataframe we need to clean up the data by doing the following:

1. Combine Neighborhoods with the same Postcode
2. Set any Neighborhood with the value of "Not assigned" to be the same as the Borough

In the code below I loop over the rows of the dataframe and create a unique mapping of each postal code. During this process I concatenate the Neighborhoods so that each unique postal code row has a string containing all the Neighborhoods associated with it.

In [13]:
init_df.groupby(['Postcode']).head()

c_data = {} # cleaned data mapping

for index, row in init_df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
    if not row['Postcode'] in c_data:
        c_data[row['Postcode']] = [row['Postcode'], row['Borough'], row['Neighborhood']]
    elif not row['Neighborhood'] in c_data[row['Postcode']][2] :
        c_data[row['Postcode']][2] += ", " + row['Neighborhood']

In [14]:
tor_df = pd.DataFrame(list(c_data.values()), columns=headers)

In [15]:
tor_df.head()

,Postcode,Borough,Neighborhood
0,M9W,Etobicoke,Northwest
1,M3L,North York,Downsview West
2,M1B,Scarborough,"Rouge, Malvern"
3,M5S,Downtown Toronto,"Harbord, University of Toronto"
4,M9P,Etobicoke,Westmount


In [16]:
tor_df.shape

(103, 3)

## PART 2: Getting Geolocations

NOTE: I tried to actually use the geocoder package and was unable to do so successfully.

In [17]:
!wget -O geospacial.csv https://cocl.us/Geospatial_data

--2019-03-25 19:46:30--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-03-25 19:46:31--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.24.197, 107.152.25.197
Connecting to ibm.box.com (ibm.box.com)|107.152.24.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-03-25 19:46:31--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [followin

In [18]:
geo_df = pd.read_csv('geospacial.csv')
geo_df = geo_df.rename(index=str, columns={'Postal Code':'Postcode'})

Now that we have the geo spacial data for the Postcodes we need to join the two dataframes together

In [19]:
tor_df.head()

,Postcode,Borough,Neighborhood
0,M9W,Etobicoke,Northwest
1,M3L,North York,Downsview West
2,M1B,Scarborough,"Rouge, Malvern"
3,M5S,Downtown Toronto,"Harbord, University of Toronto"
4,M9P,Etobicoke,Westmount


In [20]:
geo_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
tor_geo_df = pd.merge(tor_df, geo_df, on='Postcode', how='left')

In [22]:
tor_geo_df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M9W,Etobicoke,Northwest,43.706748,-79.594054
1,M3L,North York,Downsview West,43.739015,-79.506944
2,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
3,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049
4,M9P,Etobicoke,Westmount,43.696319,-79.532242


## Part 3: Analysis

In [23]:
!conda install -c conda-forge folium=0.5.0 --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  54.53 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.54 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  39.96 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  50.43 MB/s


In [24]:
import numpy as np
import matplotlib
import folium
import requests
from sklearn.cluster import KMeans

### Create new data frame using only the rows where the Borough contains the word 'Toronto'

In [25]:
only_tor_df = tor_geo_df[tor_geo_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
only_tor_df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
3,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
4,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576


Create map centered on Toronto

In [26]:
latitude = 43.653908
longitude = -79.384293
tor_map = folium.Map(location=[latitude, longitude], zoom_start=12)

In [27]:
for lat, lng, borough, neighborhood in zip(only_tor_df['Latitude'], only_tor_df['Longitude'], only_tor_df['Borough'], only_tor_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(tor_map)

In [28]:
tor_map

In [29]:
# The code was removed by Watson Studio for sharing.

### Fetch venue data from Foursquare

In [30]:
LIMIT = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = [
            'Neighborhood',
            'Neighborhood Latitude',
            'Neighborhood Longitude',
            'Venue',
            'Venue Latitude',
            'Venue Longitude',
            'Venue Category']

    return(nearby_venues)

In [31]:
venues = getNearbyVenues(names=only_tor_df['Neighborhood'],
    latitudes=only_tor_df['Latitude'],
    longitudes=only_tor_df['Longitude']
    )

Harbord, University of Toronto
Davisville North
Cabbagetown, St. James Town
Ryerson, Garden District
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Runnymede, Swansea
Little Portugal, Trinity
Studio District
Parkdale, Roncesvalles
Chinatown, Grange Park, Kensington Market
Church and Wellesley
Christie
The Danforth West, Riverdale
Harbourfront, Regent Park
Harbourfront East, Toronto Islands, Union Station
Dovercourt Village, Dufferin
First Canadian Place, Underground city
Stn A PO Boxes 25 The Esplanade
The Beaches
Brockton, Exhibition Place, Parkdale Village
Berczy Park
Lawrence Park
St. James Town
Roselawn
Moore Park, Summerhill East
High Park, The Junction South
The Beaches West, India Bazaar
Central Bay Street
Business Reply Mail Processing Centre 969 Eastern
The Annex, North Mid

In [32]:
venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbord, University of Toronto",43.662696,-79.400049,Yasu,43.662837,-79.403217,Japanese Restaurant
1,"Harbord, University of Toronto",43.662696,-79.400049,Piano Piano,43.662949,-79.402898,Italian Restaurant
2,"Harbord, University of Toronto",43.662696,-79.400049,Rasa,43.662757,-79.403988,Restaurant
3,"Harbord, University of Toronto",43.662696,-79.400049,Almond Butterfly,43.662836,-79.403365,Bakery
4,"Harbord, University of Toronto",43.662696,-79.400049,The Dessert Kitchen,43.662823,-79.402746,Dessert Shop


In [33]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,58,58,58,58,58,58
"Brockton, Exhibition Place, Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",44,44,44,44,44,44
Central Bay Street,79,79,79,79,79,79
"Chinatown, Grange Park, Kensington Market",97,97,97,97,97,97
Christie,16,16,16,16,16,16


### Apply onehot encoding to the venues category column 

In [34]:
venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

In [35]:
venues_onehot['Neighborhood'] = venues['Neighborhood']

In [36]:
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]
venues_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Get the mean of all columns when grouped by Neighborhood

In [37]:
venues_grouped = venues_onehot.groupby('Neighborhood').mean().reset_index()
venues_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.01,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0.0,0.01
1,Berczy Park,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.043478,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.058824,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.00,0.0,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00


### Find the frequent of the top 5 venues in each Neighborhood

In [38]:
num_top_venues = 5

for nh in venues_grouped['Neighborhood']:
    print("----"+nh+"----")
    temp = venues_grouped[venues_grouped['Neighborhood'] == nh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.06
1       Steakhouse  0.04
2              Bar  0.04
3  Thai Restaurant  0.04
4             Café  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2   Cheese Shop  0.03
3        Bakery  0.03
4          Café  0.03


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.09
1            Café  0.09
2     Coffee Shop  0.09
3     Yoga Studio  0.04
4       Pet Store  0.04


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0  Yoga Studio  0.06
1      Brewery  0.06
2          Spa  0.06
3  Pizza Place  0.06
4   Smoke Shop  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3          Boutique  0.07
4             Plane  0.07


---

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

### Create a dataframe to show the top 10 venues for each Neighborhood

In [40]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = venues_grouped['Neighborhood']

for ind in np.arange(venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Bar,Café,Steakhouse,Thai Restaurant,Sushi Restaurant,Asian Restaurant,Hotel,Burger Joint,Bakery
1,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Steakhouse,Italian Restaurant,Seafood Restaurant,Farmers Market,Restaurant,Pub
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Yoga Studio,Gym / Fitness Center,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Pet Store
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Garden Center,Garden,Light Rail Station,Fast Food Restaurant,Farmers Market,Moving Target,Comic Shop,Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Plane,Sculpture Garden,Boutique,Boat or Ferry,Harbor / Marina,Airport Gate,Airport


### Create clusters out of the data

In [41]:
# set number of clusters
kclusters = 10

venue_grouped_clustering = venues_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venue_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 6, 0, 0, 7, 6, 6, 0, 6, 0], dtype=int32)

### Join the tables together on the Neighborhood Column

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = only_tor_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
tor_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049,0,Café,Restaurant,Japanese Restaurant,Bakery,Bar,Bookstore,Beer Bar,Beer Store,Nightclub,Sandwich Place
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197,5,Sandwich Place,Burger Joint,Gym,Pizza Place,Park,Breakfast Spot,Hotel,Clothing Store,Food & Drink Shop,Discount Store
2,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,6,Coffee Shop,Restaurant,Bakery,Pizza Place,Chinese Restaurant,Café,Pharmacy,Pub,Italian Restaurant,Gift Shop
3,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Theater,Bubble Tea Shop,Pizza Place,Japanese Restaurant
4,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,6,Coffee Shop,Café,Hotel,American Restaurant,Seafood Restaurant,Gastropub,Deli / Bodega,Restaurant,Beer Bar,Pizza Place


### Plot the map points, and display each point with the color of it's cluster

In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters